In [ ]:
!pip install -q langchain langchain_community faiss-cpu pypdf

: 

In [ ]:
!pip install sentence-transformers


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q langchain_groq

In [ ]:
# importing libraries
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# Loading the Constitution
loader = PyPDFLoader("/content/drive/MyDrive/The_Constitution_of_Kenya_2010.pdf")
pdf = loader.load()

In [ ]:
# Chunk the text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
chunks = text_splitter.split_documents(pdf)

In [ ]:
# Embed the Chunks
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
embeds = embedding_model.embed_documents([chunk.page_content for chunk in chunks])

In [ ]:
# store in a database
from langchain.vectorstores import FAISS
db = FAISS.from_documents(chunks, embedding_model)

In [ ]:
import os
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

from langchain.chains import RetrievalQA

In [ ]:

groq_llm = ChatGroq(model_name = "moonshotai/kimi-k2-instruct-0905")

qa_chain = RetrievalQA.from_chain_type(
    llm=groq_llm,
    retriever=db.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True
)

In [ ]:
# Testing it
query = "How many elective posts do we have in Kenya ?"
result = qa_chain.invoke(query)

print("Answer:" ,result["result"])
print("Sources:" ,result["source_documents"])

In [ ]:
# Testing it
query = "What is the constitution saying about life ?"
result = qa_chain.invoke(query)

print("Answer:" ,result["result"])
print("Sources:" ,result["source_documents"])

In [ ]:
query = "How many elective posts do we have in kenya ?"
result = qa_chain.invoke(query)

print("Answer:" ,result["result"])
print("Sources:" ,result["source_documents"])